# Import Library

In [1]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score, accuracy_score
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF

import cv2

from tqdm import tqdm

import albumentations as A
from albumentations import (
    Compose, OneOf, Normalize, CenterCrop, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, RandomRotate90, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, HueSaturationValue, CoarseDropout
    )
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

import logging
import time
from contextlib import contextmanager

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Directly Settings

In [3]:
DATA_DIR = '../input/plant-pathology-2021-fgvc8/'
TEST_DIR = DATA_DIR + 'test_images/'
TRAIN_DIR = DATA_DIR + 'train_images/'
TRAIN_CSV_DIR = DATA_DIR + 'train.csv'
TEST_CSV_DIR = DATA_DIR + 'sample_submission.csv'
train_df = pd.read_csv(TRAIN_CSV_DIR)
sub_df = pd.read_csv(TEST_CSV_DIR)

In [4]:
#TARGET = train_df['labels'].unique()
TARGET = ['image','healthy', 'scab frog_eye_leaf_spot complex', 'scab', 'complex',
       'rust', 'frog_eye_leaf_spot', 'powdery_mildew',
       'scab frog_eye_leaf_spot', 'frog_eye_leaf_spot complex',
       'rust frog_eye_leaf_spot', 'powdery_mildew complex',
       'rust complex']
SUB_LABELS = ['image','labels']

onehot_df = pd.get_dummies(train_df, columns=['labels'])
col = onehot_df.drop('image', axis=1)#変換したラベルを取得する
TARGET_COLUMNS = col.columns.values
onehot_df[TARGET_COLUMNS].values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

# CFG

In [5]:
CFG = {
    'model_arch': 'efficientnet_b4',
    'img_size': 320,
    'epochs': 3,
    'lr': 0.001,
}

TARGET = ['image','healthy', 'scab frog_eye_leaf_spot complex', 'scab', 'complex',
       'rust', 'frog_eye_leaf_spot', 'powdery_mildew',
       'scab frog_eye_leaf_spot', 'frog_eye_leaf_spot complex',
       'rust frog_eye_leaf_spot', 'powdery_mildew complex',
       'rust complex']

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

@contextmanager
def timer(name, logger=None, level=logging.DEBUG):
    print_ = print if logger is None else lambda msg: logger.log(level, msg)
    t0 = time.time()
    print_(f'[{name}] start')
    yield
    print_(f'[{name}] done in {time.time() - t0:.0f} s')

# Dataset

In [6]:
class TestDataset(Dataset):
    def __init__(self, onehot_df, transform = None):
        self.onehot_df = onehot_df
        self.image_names = onehot_df["image"].values
        self.transform = transform
        
    def __len__(self):
        return len(self.onehot_df)
    
    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = TEST_DIR + image_name
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

# Transform

In [7]:
def get_transforms(*,data):
    
    if data == 'train':
        return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            RandomResizedCrop(CFG['img_size'], CFG['img_size'], scale=(0.85, 1.0)),
            HorizontalFlip(p=0.5),
            Normalize(
                mean=[0.48, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    elif data == 'valid':
        return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            #RandomResizedCrop(600, 600, scale=(0.85, 1.0)),
            #HorizontalFlip(p=0.5),
            Normalize(
                mean=[0.48, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

# Create Model

In [8]:
class EfficientNetB4(nn.Module):
    """
        Base class for all neural network modules.
        Your models should also subclass this class.
        Modules can also contain other Modules, allowing to nest them in a tree structure.
        You can assign the submodules as regular attributes:
    """
    def __init__(self):
        super().__init__()
        self.efficientnetb4 = timm.create_model(model_name = CFG['model_arch'],pretrained = False)
        in_features = self.efficientnetb4.classifier.in_features
        self.efficientnetb4.classifier = nn.Linear(in_features, len(TARGET_COLUMNS))#(input_size,output_size)
        
    def forward(self, x):
        x = self.efficientnetb4(x)
        return x
    
model = EfficientNetB4()
model = model.to(DEVICE)

# Inference 

In [9]:
model = EfficientNetB4().to(DEVICE)
model.load_state_dict(torch.load("../input/myefnet-pth/model (1).pth"))

<All keys matched successfully>

In [10]:
test_dataset = TestDataset(sub_df, transform = get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [11]:
submit_preds = []

model.eval()
with torch.no_grad():
    for X in test_loader:
        X = X.float().to(DEVICE)
        submit_preds.append(model(X).sigmoid().to("cpu"))
    submit_preds = np.concatenate([p.numpy() for p in submit_preds], axis = 0)
submit_preds

array([[0.20093137, 0.07403488, 0.01049787, 0.05088155, 0.07679804,
        0.01057071, 0.0182934 , 0.00726008, 0.00996437, 0.29756922,
        0.02956754, 0.01548253],
       [0.20358476, 0.07432201, 0.01109608, 0.05198032, 0.07863362,
        0.01110281, 0.01932468, 0.00767683, 0.01059514, 0.29770237,
        0.03031488, 0.01639665],
       [0.20451978, 0.07410364, 0.01081907, 0.04969881, 0.07917578,
        0.01095951, 0.01840554, 0.00744636, 0.01035937, 0.30359998,
        0.03030632, 0.01623542]], dtype=float32)

# Create Submission File

In [12]:
s = pd.DataFrame(submit_preds)
tmp = np.concatenate([sub_df, s], axis = 1)
tmp = pd.DataFrame(tmp)
tmp = tmp.drop([1], axis=1)
tmp.columns = TARGET
tmp = tmp.iloc[0:,1:].astype(float)
target = tmp.iloc[0:,1:].idxmax(axis = 1)
s = pd.DataFrame(target)

In [13]:
s

,0
0,rust frog_eye_leaf_spot
1,rust frog_eye_leaf_spot
2,rust frog_eye_leaf_spot


In [14]:
tmp2 = np.concatenate([sub_df, s], axis = 1)
tmp2 = pd.DataFrame(tmp2)
tmp2 = tmp2.drop([1], axis=1)
tmp2 = tmp2.set_axis(SUB_LABELS, axis = 1)

In [15]:
sub = tmp2.to_csv("submission.csv",index = False)

In [16]:
sub = pd.read_csv('./submission.csv')
sub

,image,labels
0,85f8cb619c66b863.jpg,rust frog_eye_leaf_spot
1,ad8770db05586b59.jpg,rust frog_eye_leaf_spot
2,c7b03e718489f3ca.jpg,rust frog_eye_leaf_spot
